## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation - *sur la glace ; ça pourrait permettre d'éviter les faux négatifs dûs à des variations singulier/pluriel quand on essaie d'extraire les termes qui existent déjà dans la taxonomie*)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH présents dans les données

In [1]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *
import glob

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

In [2]:
corpus_global = []
base_path = '../03-corpus/1-crawler/'
for file in listdir(base_path):
    if file.endswith('.csv'):
        with open(base_path + file, encoding = 'utf-8',) as f:
            corpus_global.append({'acteur': file[:-4], 'N_fr': 0, 'N_en' : 0}) 

In [3]:
lng = 'fr'

# À modifier pour 'lng=en'
def lire_corpus(acteur, langue=lng):
    base_path = '../03-corpus/2-data/'
    
    folder_path = path.join(base_path, '1-' + langue, acteur)
    all_files = glob.glob(path.join(folder_path, "*.csv"))
    tags = [f.split('_')[1][:-4] for f in listdir(folder_path)]

    df = DataFrame()
    for f, tag in zip(all_files, tags):
        csv = read_csv(f, encoding='utf-8', sep=',')
        #csv = csv[~csv["Address"].str.contains('pdf')] #  Problèmes 
        #csv = csv['text']
        # Using DataFrame.insert() to add a column
        #df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Title', 'text']]
        df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Address', 'Title', 'Type', 'text']].drop_duplicates(subset=['text'])
    return df

def filter_mwesw(corpus):
    file_mwesw = '../04-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' STOP ').replace('  ', " ")
    return corpus

def join(corpus):
    return " ".join(corpus)

import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG=lng)

def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))

def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append([t.split('\t')[0], t.split('\t')[1]])
        except Exception as e:
            output.append(('STOP', 'NAM'))

    return output

# from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
# lemmatizer = FrenchLefffLemmatizer()

# file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'
# with open(file_path) as f:
#    csv = read_csv(f)
# treeTag = [term for term in csv['TreeTagger'].tolist()] 
# lefff = [term for term in csv['Lefff'].tolist()]
# mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=2, max_len=8))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = tuple([t[0] for t in ng])
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

# Importer l'antidictionnaire pour filtrer les données

# Stopwords fréquents en français 
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    if lng == 'en':
        nom = 'NN'
    if lng == 'fr':
        nom = 'NOM'
    return [term for term in x if not 'STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords \
        and not 'NUM' in term[1] and term[1][0] == nom and not '.' in term[0] and not '-' in term[0] and not ':' in term[0]\
        
        # Une parenthèse fermante peut juste se trouver comme dernier token
        # Si une parenthèse est ouverte, elle doit aussi être fermée (et vice versa)
        and not ')' in term[0][:-1] and not ('(' in term[0] and not ')' in term[0]) \
        and not (')' in term[0] and not '(' in term[0])]

def filter_len(x):
    return [term for term in x if \
        (len(term[0][0]) > 2 or term[0][0] == '(')  and (len(term[0][-1]) > 2 or term[0][-1] == ')') and \
        len(term[0][0]) < 18 and len(term[0][-1]) < 18]

def filter_freq(x, freqd):
    return [term for term in x if freqd[tuple(term[0])] > 5]

file_patterns = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist()[:200] # Pour prendre les 200 structures syntaxiques les plus fréquentes dans les MeSH

def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NUM' in t[1]] # and not 'NOM NOM' in t[1]

def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 


def llr_ngrammes(terms, freq, fdtok, patterns):
    llr = []

    ngrammes = [term[0] for term in terms]
        
    for t in ngrammes:
        if len(t) == 1:
            try:
                llr.append({'Terme' : str(t[0]), 'Structure syntaxique': patterns[t], 'Fréquence (TF)' : fdtok[str(t[0])], 'LLR': '-', 'p-value': '-'})
            except Exception as e:
                print(t, str(e))
        else:
            c_prior = freq[t[:-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fdtok[t[-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = freq[t]             # Le ngramme lui-même P(w1w2w3..wn)

            try:
                res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
                p_value = chi2.sf(res, 1) # 1 degrees of freedom

                if p_value < 0.001 or (res == float('-inf')):
                    #llr.append({'Collocation' : " ".join(t).replace("' ", "'").replace("( ", "(").replace(" )", ")"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p_value})
                    llr.append({'Terme' : t, 'Structure syntaxique': patterns[t], 'Fréquence (TF)' : c_ngram, 'LLR': res, 'p-value': p_value})
            
            except Exception as e:
                print(t, str(e))
            
    return llr

def join_term(x):
    if type(x) == tuple:
        return " ".join(x).replace("' ", "'").replace("( ", "(").replace(" )", ")")
    else:
        return x

import xml.etree.ElementTree as ET
import re
import pandas as pd
base_path = '../04-filtrage/MeSH/'

tree = ET.parse(base_path + 'fredesc2019.xml')
root = tree.getroot()

def flatten(l):
    return [item for sublist in l for item in sublist]

data_mesh = [{'mesh_id' : x.find('DescriptorUI').text.strip('\n'), \
         'label_fr' : x.find('DescriptorName').find('String').text.split('[')[0], \
         'label_en' : x.find('DescriptorName').find('String').text.split('[')[1].strip(']'), \
         'synonymes (en/fr)' : flatten([[term.find('String').text for term in concept.find('TermList').findall('Term')] for concept in x.find('ConceptList').findall('Concept')]) \
         } for x in root.findall('DescriptorRecord')]

c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:739: FutureWarning: Possible nested set at position 8
  punct2find_re = re.compile("([^ ])([[" + ALONEMARKS + "])",
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2043: FutureWarning: Possible nested set at position 152
  DnsHostMatch_re = re.compile("(" + DnsHost_expression + ")",
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2067: FutureWarning: Possible nested set at position 409
  UrlMatch_re = re.compile(UrlMatch_expression, re.VERBOSE | re.IGNORECASE)
c:\Users\p1115145\AppData\Local\Programs\Python\Python310\lib\site-packages\treetaggerwrapper.py:2079: FutureWarning: Possible nested set at position 192
  EmailMatch_re = re.compile(EmailMatch_expression, re.VERBOSE | re.IGNORECASE)


In [4]:
for x in range(len(corpus_global)-5) : 
    acteur = corpus_global[x]['acteur']

    data = lire_corpus(acteur) 
    nb_docs = len(data)
    print(acteur, str(nb_docs))
    corpus_global[x]['N_fr'] = nb_docs

    base_path = '../03-corpus/2-data/'
    folder_path = path.join(base_path, '1-' + lng, acteur)

    #data.to_csv(folder_path + '.csv')

    #Nettoyage
    punct = '[!#$%&•►*+,;\/\\<=>?@[\]^_{|}~©«»—“”–—]'
    spaces = '\s+'
    postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
    # phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

    text = [str(t).strip('\n').lower().replace('’', '\'') for t in data['text'].tolist()]
    text = [re.sub(spaces, ' ', t) for t in text]
    text = [re.sub(postals, ' STOP ', t) for t in text]
    text = [re.sub(punct, ' STOP ', t) for t in text]
    text = [t.replace("  ", " " ) for t in text]

    corpus = join(text)
    corpus = filter_mwesw(corpus)
    

    # Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour voir combien de temps ça devrait 
    # prendre au Tree Tagger pour tokeniser et tagger notre corpus
    tok(corpus)

    tagged = tagging(corpus)
    tokens = [t[0] for t in tagged]

    # df_tagged = DataFrame(tagged, columns=['Token', 'Tag TreeTagger'])
    # df_tagged['Tag Lefff'] = df_tagged['Tag TreeTagger'].map(mapping)
    # df_tagged['Lemme'] = df_tagged['Token'] # S'il est pas capable de lemmatiser ensuite, on garde l'expression telle quelle

    # tagged_dict = df_tagged.to_dict('records')

    # # tokens = df_tagged['Token'].tolist()
    # # tags = df_tagged['Tag Lefff'].tolist()
    # # tagged_lefff = [(t, tg) for t,tg in zip(tokens, tags)]

    # for term in tagged_dict:
    #     if lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff']) == []:
    #         term_l = lemmatizer.lemmatize(word=term['Token'])
        
    #     elif type(lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])) == str:
    #         term_l  = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])

    #     else:
    #         term_l = lemmatizer.lemmatize(word=term['Token'], pos=term['Tag Lefff'])[0][0]

    #     term['Lemme'] = term_l

    # tagged = DataFrame(tagged_dict)
    # tagged = tagged.drop(columns=['Tag Lefff']).values.tolist()


    # Extraction de n-grammes
    ngrammes = extr_ngrams(tagged)
    #ngrammes_lem = extr_ngrams(tagged_lem)


    # Extraction de patrons syntaxiques
    phrases = extract_patterns(ngrammes)
    # phrases_lem = extract_patterns(ngrammes_lem)

    # def freq(phrases):
    #     return FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])

    # def freq(phrases):
    #     return FreqDist([t[0] for t in phrases])

    frequencies = FreqDist(everygrams(tokens, min_len=2, max_len=8))

    # Filtrage (antidictionnaire) 
    phrases = filtrer_stopwords(phrases)

    # Filtrage (mots <2 caractères ou >18 caractères)
    phrases = filter_len(phrases)

    # Filtrage (fréquence > 5)
    phrases = filter_freq(phrases, frequencies)

    # Chunking
    #phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
    phrases = [[term[0], " ".join(term[1])] for term in phrases]
    # phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

    for phrase in phrases:
        phrase.append(frequencies[tuple(phrase[0])])

    # Filtrage (patrons syntaxiques)
    terms = filter_patterns(phrases)

    print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
    print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))


    # def extract_terms(liste_terms):
    #     file_path = '../04-filtrage/output/'
    #     tab = DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
    #     tab.sort_values(["Fréquence"], 
    #                         axis=0,
    #                         ascending=[False], 
    #                         inplace=True)

    #     return 

    #     # file_path = path.join(file_path, tag, tag)                    
    #     # tab.to_csv(file_path + '_terms.csv')

    # extract_terms(terms)
    # #extract_terms(terms_lemmatized)

    for phrase in terms:
        phrase[0] = tuple(phrase[0])


    terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
    terms_patterns = terms_patterns.to_dict('records')
    dict_patterns = {}
    for term in terms_patterns:
        exp = term['Expression']
        pattern = term['Structure syntaxique']
        dict_patterns[exp] = pattern

    # Collocations significatives (log-likelihood ratio)
    N = len(tokens)
    fd_tokens = nltk.FreqDist(tokens)

    significant_coll = llr_ngrammes(terms, frequencies, fd_tokens, dict_patterns)
    #terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms] # Si on veut pas appliquer le LLR
        
    print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len(terms) - len(significant_coll)))

    df = DataFrame(significant_coll).sort_values(by = "Fréquence (TF)", ascending=False).drop_duplicates()

    # On veut faire join pour tous les termes[collocation]
    df['Terme'] = df['Terme'].apply(lambda x: join_term(x))

    if lng == 'en':
        acteur = acteur + '_' + lng

    df = df.drop(columns=['p-value'])

    # Filtrage (fréquence documentaire)
    dfs = {term: len([doc for doc in text if term in doc]) for term in df['Terme'].tolist()}
    max_df = round(0.98 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 98% des documents 
    min_df = round(0.01 * nb_docs)         # Pour rejeter les termes qui se retrouvent dans moins de 1% des documents

    dfs = {term:df for term,df in dfs.items() if df < max_df and df > min_df} # Si df = 0, c'est un artefact créé par le prétraitement lui-même
    zeros =  {term:df for term,df in dfs.items() if df == 0}  
    print('N(df == 0) : '+ str(len(zeros)))
    
    dfs = DataFrame(list(dfs.items()),columns = ['Terme','Fréquence documentaire (DF)']) 
    df = df.merge(dfs, on='Terme').drop_duplicates()

    df.sort_values(['Fréquence (TF)'], 
                axis=0,
                ascending=[False], 
                inplace=True)

    list_terms = df['Terme'].tolist()

    # Détection de termes appartenant aux MeSH
    df['isMeSHTerm']= False # On set à False puis on va changer pour True si on trouve le terme
    df['MeSHID'] = None
    df['MesH_prefLabel_fr'] = None
    df['MesH_prefLabel_en'] = None

    from nltk.tokenize import MWETokenizer
    file_path = '../04-filtrage/MeSH/mesh-fr.txt'

    with open (file_path, 'r', encoding='utf-8') as f:
        mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
        tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
        mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
        mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
        #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

    extr_mesh = tokenizer_mesh.tokenize(list_terms)

    for t in extr_mesh:
        if t in mesh:
            df.loc[df['Terme'] == t, 'isMeSHTerm'] = True

    # Extraction de termes existant dans la taxonomie
    df['isTaxoTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme

    file_path = '../04-filtrage/default_taxo_labels.csv'

    with open(file_path, 'r', encoding='utf-8') as f:
        default = read_csv(f, sep=';')
        taxo_terms = list(dict.fromkeys([str(t).strip().lower() for t in default['Label'].tolist()]))

    for t in df['Terme'].tolist():
        if t in taxo_terms:
            df.loc[df['Terme'] == t, 'isTaxoTerm'] = True

    # Mapping to MeSH ids
    for t in df[df['isMeSHTerm'] == True]['Terme'].tolist():
        for d in data_mesh:
            if t in [str(x).lower() for x in d['synonymes (en/fr)']]:
                df.loc[df['Terme'] == t, 'MeSHID'] = d['mesh_id']
                df.loc[df['Terme'] == t, 'MesH_prefLabel_fr'] = d['label_fr']
                df.loc[df['Terme'] == t, 'MesH_prefLabel_en'] = d['label_en']

    df = df[['Terme', 'Structure syntaxique',	'Fréquence (TF)', 'Fréquence documentaire (DF)', 'LLR', 'isMeSHTerm', 'isTaxoTerm', 'MeSHID', 'MesH_prefLabel_fr', 'MesH_prefLabel_en']]
    df.insert(0, 'Corpus', acteur)

    if lng == 'en':
        acteur = acteur + '_' + lng

    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 
    #df.to_csv(output_path)

    print('On a fini avec ce corpus : {}.'.format(acteur))

chum 2238
Avec le RegExpTokenizer, notre corpus contient 870350 tokens.
Le POS tagging devrait prendre environ 1 minutes.
Avant filtrage, on a 6378708 ngrammes.


MemoryError: 